In [ ]:
%%capture
!pip install evaluate
!pip install transformers

In [ ]:
import subprocess

def get_gpu_name():
    try:
        output = subprocess.check_output(['nvidia-smi', '--query-gpu=name', '--format=csv,noheader'], encoding='utf-8')
        gpu_name = output.strip()
        return gpu_name
    except (subprocess.CalledProcessError, FileNotFoundError):
        return None
print(get_gpu_name())

Tesla T4


In [ ]:
import pandas as pd
import numpy as np
import evaluate
import re

from enum import Enum
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForPreTraining
from transformers import DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import EarlyStoppingCallback, IntervalStrategy

from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/violence/violence-v1.csv")

In [ ]:
print(len(df))
df = df[df['text'].notna()]
print(len(df))

8148
8147


In [ ]:
class Annotation:
    def __init__(self, text, religio, ethno, nondenominational, noncommunal):
        self.text = text
        self.religio = religio
        self.ethno = ethno
        self.nondenominational = nondenominational
        self.noncommunal = noncommunal

In [ ]:
def categorize_violence(violences, text):
    ore = [violences]
    violences = str(violences)
    violences = re.findall(r'\d',violences)
    return_list = list()
    for v in violences:
        if int(v)!=0:
            return_list.append(int(v))
    if(len(return_list)>0):
        return True
    return False

In [ ]:
annotation_list = []
for idx, row in df.iterrows():
    text = row.text
    annotation = Annotation(
        row.text,
        categorize_violence(row['religio-communal'],text),
        categorize_violence(row['ethno-communal'], text),
        categorize_violence(row['other-communal'], text),
        categorize_violence(row['generic-violence'], text)
    )
    annotation_list.append(annotation)

In [ ]:
df = pd.DataFrame([x.__dict__ for x in annotation_list])

In [ ]:
from datasets import DatasetDict

dataset = Dataset.from_pandas(df)
train_test_valid = dataset.train_test_split(test_size=0.4)
test_valid = train_test_valid['test'].train_test_split(test_size=0.5)

dataset = DatasetDict({
    'train': train_test_valid['train'],
    'test': test_valid['test'],
    'valid': test_valid['train']})
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'religio', 'ethno', 'nondenominational', 'noncommunal'],
        num_rows: 4888
    })
    test: Dataset({
        features: ['text', 'religio', 'ethno', 'nondenominational', 'noncommunal'],
        num_rows: 1630
    })
    valid: Dataset({
        features: ['text', 'religio', 'ethno', 'nondenominational', 'noncommunal'],
        num_rows: 1629
    })
})


In [ ]:
labels = [label for label in dataset['train'].features.keys() if label not in ['text']]
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}
labels

['religio', 'ethno', 'nondenominational', 'noncommunal']

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/banglabert")
model = AutoModelForPreTraining.from_pretrained("csebuetnlp/banglabert")

In [ ]:
def preprocess_data(examples):
  # take a batch of texts
  text = examples["text"]
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=512)
  # add labels
  labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(text), len(labels)))
  # fill numpy array
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]

  encoding["labels"] = labels_matrix.tolist()
  
  return encoding

In [ ]:
encoded_dataset = dataset.map(preprocess_data, batched=True, remove_columns=dataset['train'].column_names)

example = encoded_dataset['train'][0]
print(example.keys())

Map:   0%|          | 0/4888 [00:00<?, ? examples/s]

Map:   0%|          | 0/1630 [00:00<?, ? examples/s]

Map:   0%|          | 0/1629 [00:00<?, ? examples/s]

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])


In [ ]:
encoded_dataset.set_format("torch")
encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 4888
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 1630
    })
    valid: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 1629
    })
})

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    "csebuetnlp/banglabert", 
    problem_type="multi_label_classification", 
    num_labels=len(labels), 
    id2label=id2label, 
    label2id=label2id
)

Some weights of the model checkpoint at csebuetnlp/banglabert were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at csebuetnlp/banglabert and are newly initialized: ['classifier.out_proj.weight', 'classi

In [ ]:
metric_name = "f1"

training_args = TrainingArguments(
    output_dir="violence",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=30,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model='f1',
)

In [ ]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch
    
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]
)

trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a ElectraTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,No log,0.256129,0.470405,0.665641,0.619632
2,0.286100,0.234332,0.526937,0.696468,0.637423
3,0.286100,0.250677,0.606324,0.762725,0.664417


Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,No log,0.256129,0.470405,0.665641,0.619632
2,0.286100,0.234332,0.526937,0.696468,0.637423
3,0.286100,0.250677,0.606324,0.762725,0.664417
4,0.186400,0.284951,0.617377,0.797525,0.627607
5,0.127300,0.326504,0.522124,0.694357,0.641104
6,0.127300,0.359797,0.608123,0.788121,0.621472
7,0.077700,0.411344,0.563399,0.730933,0.643558


TrainOutput(global_step=2142, training_loss=0.16233879199771542, metrics={'train_runtime': 3582.0682, 'train_samples_per_second': 40.937, 'train_steps_per_second': 2.563, 'total_flos': 9002769531764736.0, 'train_loss': 0.16233879199771542, 'epoch': 7.0})

In [ ]:
trainer.evaluate(encoded_dataset["test"])

{'eval_loss': 0.28495144844055176,
 'eval_f1': 0.6173772261198057,
 'eval_roc_auc': 0.7975247370111944,
 'eval_accuracy': 0.6276073619631902,
 'eval_runtime': 58.414,
 'eval_samples_per_second': 27.904,
 'eval_steps_per_second': 1.746,
 'epoch': 7.0}

In [ ]:
trainer.save_model("/content/drive/MyDrive/violence/violence-bert-four")